# Table of Content:-
* **Panda** 
* **Numpy** 
* **Matplotlib**
* **Seaborn**
* **Correlation Matri**x
* **Heatmap**
* **ROC-Curv**e
* **Precision Curve** 
* **Logistic Regression** 
* **XGBoost Classifier**
* **Random Forest Classifie**r
* **Conclusion**

# Dataset -Description
* **Age**- Age of the patients those are sufferening from the Cancer.
* **Operation-Year**- When the operation started 
* **Positive_lymph_node**- Positive Lymph Node Ratio as an Indicator of Prognosis and Local Tumor Clearance in N3 Gastric Cancer
* **survival_status**- It is binary class. 1 means "**Survive**" and 0 means "**Dead**"

In [ ]:
from IPython.display import Image
Image("../input/cancer/breast.png", width=1000, height=1000)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve


In [ ]:
df=pd.read_csv('../input/habermans-survival-data-set/haberman.csv')
df.head()

**Observation(s)**:

* Data contains values and columns name are missing in the dataset.
* Need to add column data in dataset.
* Column names can be found in column metadata tab in Kaggle.

**name the columns as given below**

In [ ]:
df.columns=['Age','Operation_Year','Positive_lymph_node','survival_status']
df.head()

In [ ]:
df['survival_status'].value_counts()

* Converting the Possitive as 1 and negative as 0
* 1 as 1 and 2 as 0

In [ ]:
survival_status={1:1,2:0}
df['survival_status']=[survival_status[x] for x in df['survival_status']]

In [ ]:
df['survival_status'].value_counts()

In [ ]:
df.describe()

In [ ]:
sns.boxplot(x='survival_status',y='Age', data=df, linewidth=0.35)

In [ ]:
df['survival_status'].unique()

In [ ]:
sns.pairplot(data=df)
plt.show()

In [ ]:
df['survival_status'].unique()

**When lymph nodes are free, or clear, of cancer, the test results are negative. If lymph nodes have some cancer cells in them, they are called positive. Your pathology report will tell you how many lymph nodes were removed, and of those, how many tested positive for the presence of cancer cells**

**Objective:**
To predict whether a patient will survive after 5 years or not based upon the patient's age,his/her operation_year and the number of positive lymph nodes

**BI-VARIATE ANALYSIS**

**2-D Scatter Plot**

In [ ]:
df.plot(x='Age', y='Positive_lymph_node', kind='scatter')
plt.show()

**Obs**
Survival rate is hight as lmph node is zero

2-D Scatter plot with color-coding for each survival status type/class.

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(df, hue="survival_status",size=6).map(plt.scatter, "Age","Positive_lymph_node").add_legend()
plt.show()

**Observations**:

* It seems most of the patients have 0 positive lypmh nodes detected.
* Here we clearly see that blue points are not seperated from orange points.
* So, by looking this 2-D scatter plot between 'Age' and 'positive_lymph_nodes' we cannot make any decision regarding patient's survival.
* Therefore, we have to check all combination/pair of features to make good classification/decision.
* Number of Combinations of features : 3C2 = 3 (excluding class-attribute 'Survival')
* Now, for these combination to analyse,Pair-Plot concept is used

**Pair Plot**

In [ ]:
sns.set_style('whitegrid')
sns.pairplot(data=df)
plt.show()

**Age Vs Possitive Lymph Node**

**UNIVARIATE ANALYSIS**

Histogram, PDF

In [ ]:
sns.FacetGrid(df, hue="survival_status",size=5).map(sns.distplot,"Positive_lymph_node").add_legend()
plt.show()

In [ ]:
sns.FacetGrid(df, hue='survival_status').map(sns.histplot, "Positive_lymph_node").add_legend()
plt.show()

In [ ]:
sns.FacetGrid(df, hue="survival_status",size=5).map(sns.distplot,"Operation_Year").add_legend()
plt.show()

# Observation:

* From the above PDFs(Univariate analysis) both Age and Operation_Year are not good features for useful insights as the distibution is more similar for both people who survived and also dead.
* 
* positive_lymph_nodes is the only feature that is useful to know about the survival status of patients as there is difference between the distributions for both classes(labels). From that distibution we can infer that most survival patients have fallen in to zero positive_lymph_nodes.
* 
* From the year distribution, we can observe that people who didnt survive suddenly rise and fall in between 1958 and 1960. More number of people are not survived in year of operation of 1965

# Divide the data set in two according to the label Survival status 
# Alive means status=1 and dead means status =2

In [ ]:
alive=df.loc[df['survival_status']==1]
dead=df.loc[df['survival_status']==0]

In [ ]:
count,bin_edges=np.histogram(alive['Positive_lymph_node'], bins=10, density=True)
pdf=count/sum(count)

print(pdf)
print(bin_edges)
cdf=np.cumsum(pdf)

plt.plot(bin_edges[1:],pdf)
plt.plot(bin_edges[1:], cdf)
plt.legend(['Pdf for the patients who survive more than 5 years',
            'Cdf for the patients who survive more than 5 years'])
            

In [ ]:
count,bin_edges=np.histogram(dead['Positive_lymph_node'], bins=10, density=True)
pdf=count/sum(count)

print(pdf)
print(bin_edges)
cdf=np.cumsum(pdf)

plt.plot(bin_edges[1:],pdf)
plt.plot(bin_edges[1:], cdf)
plt.legend(['Pdf for the patients who survive more than 5 years',
            'Cdf for the patients who survive more than 5 years'])
            

# Mean, Variance and Std-dev 

In [ ]:
df.describe()

# Alive statics

In [ ]:
print("Summary Statistics of Patients who are alive within 5 years:")
alive.describe()

# Dead statics

In [ ]:
print("Summary Statistics of Patients who are dead within 5 years:")
dead.describe()

# Observations:

From both the tables we can observe that almost for all the features the statistics are similar except for positive_lymph_nodes.

The mean(average) of positive_lymph_nodes is more for people who died within 5 years than people who have survived for more than 5 years

From the observation of CDFs, we can infer that patients above 46 axillary nodes detected can be considered as dead within 5 years. So,People having less number of positive_lymph_node have survived.

# Median, Percentile, Quantile, IQR, MAD

In [ ]:
#Median, Quantiles, Percentiles, IQR.
print("\nMedian")
print(np.median(alive['Positive_lymph_node']))
#Median with an outlier
print(np.median(np.append(alive['Positive_lymph_node'],50)))

print(np.median(dead['Positive_lymph_node']))

print("\nQuatitle")

print(np.percentile(alive['Positive_lymph_node'], np.arange(0,100,0.25)))
print(np.percentile(dead['Positive_lymph_node'],np.arange(0,100,0.25)))

print("\n90thpercentile")
print(np.percentile(alive['Positive_lymph_node'],90))
print(np.percentile(dead['Positive_lymph_node'],90))



In [ ]:
from statsmodels import robust
print("\nMedian Absoute Deviation")
print(robust.mad(alive['Positive_lymph_node']))
print(robust.mad(dead['Positive_lymph_node']))

**The median absolute deviation is a measure of statistical dispersion. Moreover, the MAD is a robust statistic, being more resilient to outliers in a data set than the standard deviation. In the standard deviation, the distances from the mean are squared, so large deviations are weighted more heavily, and thus outliers can heavily influence it. In the MAD, the deviations of a small number of outliers are irrelevant.**

# Box plot and Whiskers

"""
Box plot takes a less space and visually represents the five number summary of the data points in a box. 
The outliers are displayed as points outside the box.
1. Q1 - 1.5*IQR
2. Q1 (25th percentile)
3. Q2 (50th percentile or median)
4. Q3 (75th percentile)
5. Q3 + 1.5*IQR
Inter Quartile Range = Q3 -Q1

In [ ]:
sns.boxplot(x='survival_status',y='Positive_lymph_node', data=df)
plt.plot()

In [ ]:
sns.boxplot(x='survival_status', y='Age', data=df)
plt.show()

In [ ]:
sns.boxplot(x='survival_status', y='Operation_Year', data=df)
plt.show()

# Violin Plot

In [ ]:
sns.violinplot(x='survival_status', y='Positive_lymph_node', data=df)
plt.show()

In [ ]:
sns.violinplot(x='survival_status', y='Age', data=df)
plt.show()

In [ ]:
sns.violinplot(x='survival_status', y='Operation_Year', data=df)
plt.show()

In [ ]:
fig , axes=plt.subplots(1,3,figsize=(15,5))
for idx,feature in enumerate(list(df.columns)[:-1]):
    sns.violinplot(x='survival_status', y=feature, data=df,ax=axes[idx])
plt.show()

> Observation:

* The number of positive lymph nodes of the survivors is highly densed from 0 to 5.
* Almost 80% of the patients have less than or equal to 5 positive lymph survived more than 5 years.
* From box plots and violin plots, we can say that more no of patients who are dead have age between 46-62,year between 59-65 and the patients who survived have age between 42-60, year between 60-66.

# Contour plot

In [ ]:
sns.jointplot(x='Age',y='Operation_Year', data=df, kind="kde")
plt.plot()

# Observation: There are more number of people who have undergone operation during the year 1959 - 1964 period and between the ages 42 - 60.



# Correlation Matrix

In [ ]:
# Generate and visualize the correlation matrix
corr = df.corr().round(2)

# Mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set figure size
f, ax = plt.subplots(figsize=(20, 20))

# Define custom colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.tight_layout()

# Heat Map

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), cmap='Blues', annot=True)
plt.tight_layout()

In [ ]:
df['survival_status'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x=df['survival_status'])
plt.show()

In [ ]:
X=df.drop(['survival_status'], axis=1)
y=df['survival_status']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=7)

# Logistic Regression 

In [ ]:
from IPython.display import Image
Image("../input/logistic-regression/logistic.jpg")

In [ ]:
Image("../input/logistic-regr/logisticregression.png")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
lr=LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_train)

In [ ]:
from sklearn import metrics

# Metrics calculation 

In [ ]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
sns.distplot(y_train-y_pred)
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Histogram of residula")
plt.show()

In [ ]:
# Predicting the Test data with model 
y_test_pred=lr.predict(X_test)

In [ ]:
lr_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",lr_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
cfm=confusion_matrix(y_test, y_test_pred)
trueNegative=cfm[0][0]
falsePossitive=cfm[0][1]
false_negative=cfm[1][0]
truePositive=cfm[1][1]

print("Confusion Matrix", cfm)
print("true negative", trueNegative)
print("False Positive", falsePossitive)
print("false Negative", false_negative)
print("True Positive", truePositive)


In [ ]:
print("correct prediction", 
      round((trueNegative+truePositive)/len(y_test_pred)*100, 1),'%')

# Confusion Matrix

In [ ]:
cfm_df=pd.DataFrame(cfm, range(2), range(2))
plt.figure(figsize=(10,10))
sns.heatmap(cfm_df, cmap='Reds', annot=True)
plt.show()

In [ ]:
pd.crosstab(y_test, y_test_pred, rownames=['True'], colnames=['Predicted'], margins=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=lr.predict_proba(X_test)[:,1]


In [ ]:
y_test_pred_prob

In [ ]:
from sklearn.metrics import roc_curve


In [ ]:
metrics.roc_auc_score(y_test, y_test_pred_prob)

In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

# Recall-Precision Curve 
* Precision-Recall Curves and AUC
* It is calculated as the number of true positives divided by the total number of true positives and false positives.
* >Precision = TruePositives / (TruePositives + FalsePositives) Recall = TruePositives / (TruePositives + FalseNegatives)

In [ ]:
no_skill=len(y==1)/len(y)
y_test_prob=lr.predict_proba(X_test)[:,1]
plt.figure(figsize=(10,8))
plt.plot([0,1],[no_skill, no_skill], label="No Skill")
precision, recall,_ =precision_recall_curve(y_test, y_test_prob)
plt.plot(recall, precision, marker='',label="Logistic Regression")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve")
plt.legend()
plt.show()

# XGBoost classification

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train, y_train)

In [ ]:
y_pred=xgb.predict(X_train)

In [ ]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
# Predicting the Test data with model 
y_test_pred=xgb.predict(X_test)

# Mode Evaluation of Test dataset

In [ ]:
xgb_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",xgb_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
cfm=confusion_matrix(y_test, y_test_pred)
trueNegative=cfm[0][0]
falsePossitive=cfm[0][1]
false_negative=cfm[1][0]
truePositive=cfm[1][1]
print("Confusion Matrix", cfm)
print("true negative", trueNegative)
print("False Positive", falsePossitive)
print("false Negative", false_negative)
print("True Positive", truePositive)


In [ ]:
print("correct prediction", 
      round((trueNegative+truePositive)/len(y_test_pred)*100, 1),'%')

In [ ]:
z_df=pd.DataFrame(cfm, range(2), range(2))
plt.figure(figsize=(10,10))
sns.heatmap(z_df, cmap='Reds', annot=True)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
y_test_pred_prob=xgb.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='XGBoost')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC ")
plt.show()

# Precision Recall Curve

In [ ]:
no_skill=len(y==1)/len(y)
y_test_prob=xgb.predict_proba(X_test)[:,1]
plt.figure(figsize=(10,8))
plt.plot([0,1],[no_skill, no_skill], label="No Skill")
precision, recall,_ =precision_recall_curve(y_test, y_test_prob)
plt.plot(recall, precision, marker='',label="XGBoost Classifier")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve")
plt.legend()
plt.show()

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train, y_train)

In [ ]:
y_pred=rfc.predict(X_train)

In [ ]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
# Predicting the Test data with model 
y_test_pred=rfc.predict(X_test)

# Model Evaluation for Test dataset

In [ ]:
rfc_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",rfc_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
cfm=confusion_matrix(y_test, y_test_pred)
trueNegative=cfm[0][0]
falsePossitive=cfm[0][1]
false_negative=cfm[1][0]
truePositive=cfm[1][1]
print("Confusion Matrix", cfm)

print("true negative", trueNegative)
print("False Positive", falsePossitive)
print("false Negative", false_negative)
print("True Positive", truePositive)


In [ ]:
print("correct prediction", 
      round((trueNegative+truePositive)/len(y_test_pred)*100, 1),'%')

cfm_df=pd.DataFrame(cfm, range(2), range(2))
plt.figure(figsize=(10,10))
sns.heatmap(cfm_df, cmap='Reds', annot=True)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
cfm_df=pd.DataFrame(cfm, range(2), range(2))
plt.figure(figsize=(10,10))
sns.heatmap(cfm_df, cmap='Reds', annot=True)

In [ ]:
y_test_pred_prob=xgb.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)
plt.figure(figsize=(10,10))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr, tpr, label='Random Forest')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC ")
plt.show()

In [ ]:
no_skill=len(y==1)/len(y)
y_test_prob=rfc.predict_proba(X_test)[:,1]
plt.figure(figsize=(10,8))
plt.plot([0,1],[no_skill, no_skill], label="No Skill")
precision, recall,_ =precision_recall_curve(y_test, y_test_prob)
plt.plot(recall, precision, marker='',label="Random Forest Classifier")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve")
plt.legend()
plt.show()

In [ ]:
print("Logistic Regression", lr_acc*100)
print("XGBosst", xgb_acc*100)
print("Random Forest", rfc_acc*100)

# Conclusions:
* Logistic Regression works well with this kind of the dataset
* There are 306 observations with 4 features in the data set.
* It is an imbalanced dataset with-
*            a. 225 patients belonging to status 1, those who survived for 5 years and longer and
*            b. 81 patients belonging to status 2, those who survived for less than 5 years.
* Using scatter plot(Bi-variate analysis) -
*            a. Most of the people have zero positive_lymph_nodes.
*            b. We cannot distinguish between the people who survived and who didn't survive.
* Using Pair-plot concept(Bi-variate analysis)-
*            a. positive_lymph_nodes VERSUS Age is the useful plot to atleast get the insight that most people who survived have 0 postive lymph nodes detected.
*            b. Age and Operation_Year have overlapping curves which makes difficult for classifying the survival status.
*            c. but we cannot distinguish the data easily with the help of these plots as most of them are overlapping.                          
* Using PDFs(Uni-variate Analysis)-
*            a.  both Age and Operation_Year are not good features for useful insights as the distibution is more similar for both people who survived and also dead.
*            b. positive_lymph_nodes is the only feature that is useful to know about the survival status of patients as there is difference between the distributions for both classes(labels). From that distibution we can infer that most survival patients have fallen in to zero positive_lymph_nodes.
*            c. More number of people are not survived in year of operation of 1965.
* Using CDFs(Uni-variate analysis)-
*            a. We can observe that almost for all the features the statistics are similar except for positive_lymph_nodes.
*            b. We can infer that patients above 46 axillary nodes detected can be considered as dead within 5 years. So,People having less number of positive_lymph_node have survived over 5 years.
*         The mean(average) of positive_lymph_nodes is more for people who died within 5 years than people who have survived for more than 5 years.
* Mean age of patients who survived is 52 years and who didn't survive is 54 years.
* Using Box plot and Violin plots-
*            a. The number of positive lymph nodes of the survivors is highly densed from 0 to 5.
*            b. Almost 80% of the patients have less than or equal to 5 positive lymph survived more than 5 years.
*            c. From box plots and violin plots, we can say that more no of patients who are dead have age between 46-62,year between 59-65 and the patients who survived have age between 42-60, year between 60-66. 
* Using Contour plot-
*            a. There are more number of people who have undergone operation during the year 1959 - 1964 period and between the ages 42 - 60.
* If you like you can follow and review my other kernels. Here is my profile
[Rabi SIngh](https://www.kaggle.com/jurk06/code)

In [ ]:
from IPython.display import Image
Image("../input/thank-you/download.jpg", width=1000, height=1000)